### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** 25 ноября 2019, 15:00   
**Штраф за опоздание:** -2 балла после 15:00 25 ноября, -4 балла после 15:00 2 декабря, -6 баллов после 15:00 9 декабря  -8 баллов после 15:00 16 декабря.

При отправлении ДЗ указывайте фамилию в названии файла Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:
[ML0919, Задание 3] Фамилия Имя. 


Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (3 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на [wine](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html) и [Speed Dating Data](https://cloud.mail.ru/public/8nHV/p6J7wY1y1)

###### Задание 2 (3 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine и Speed Dating Data. 
Для этого используем numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw3.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%load_ext pycodestyle_magic

In [2]:
#%%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None,
                 sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return (l_s * (1 - np.sum(
            (l_c / l_s) ** 2, axis=1)).reshape(-1, 1) +
                r_s * (1 - np.sum(
            (r_c / r_s) ** 2, axis=1)).reshape(-1, 1)) / (l_s + r_s)

    def __entropy(self, l_c, l_s, r_c, r_s):
        return (l_s * (-np.sum((l_c / l_s) * np.log(
                (l_c + 1e-6)/ l_s), axis=1)).reshape(-1, 1) +
                r_s * (-np.sum((r_c / r_s) * np.log(
                (r_c + 1e-6)/ r_s), axis=1)).reshape(-1, 1)) / (l_s + r_s)

    def __misclass(self, l_c, l_s, r_c, r_s):
        return (l_s * (1 - np.max(
                    l_c / l_s, axis=1)).reshape(-1, 1) +
                r_s * (1 - np.max(
                    r_c / r_s, axis=1)).reshape(-1, 1)) / (l_s + r_s)

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.sqrt(n_feature))]

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.log2(n_feature))]

    def __get_feature_ids_N(self, n_feature):
        return range(n_feature)

    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        x_s, y_s = self.__sort_samples(x, y)
        splt = int(self.min_samples_split / 2 - 1)
        if splt:
            y_s = y_s[splt:-splt]
        border = np.where(y_s[:-1] != y_s[1:])[0] + (splt + 1)

        if len(border) == 0:
            return np.inf, None
        
        shift = [splt] + list(border[:-1])
        template = np.zeros((border.shape[0], self.num_class))
        template[list(range(border.shape[0])), y_s[border - 1]] = 1

        matrix_counts = template * (border - shift).reshape(-1, 1)
        matrix_counts[0] += np.bincount(y_s[:splt], minlength=self.num_class)

        l_c = np.cumsum(matrix_counts, axis=0)
        r_c = np.bincount(y_s, minlength=self.num_class) - l_c
        l_s = border.reshape(l_c.shape[0], 1)
        r_s = y_s.shape[0] - l_s
        
        G_res = self.G_function(l_c, l_s, r_c, r_s)
        idx = np.argmin(G_res)
        indx = l_s[idx][0]

        return G_res[idx], (x_s[indx - 1] + x_s[indx]) / 2.

    def __fit_node(self, x, y, node_id, depth):
        if (self.max_depth is not None and depth >= self.max_depth) or\
                y.size < self.min_samples_split or\
                np.unique(y).shape[0] == 1 or\
                np.bincount(y).argmax() >= (y.size * self.sufficient_share):
            #Set leaf class
            self.tree[node_id] = (self.LEAF_TYPE, np.bincount(y).argmax())
        else:
            thresholds = np.array(
                [self.__find_threshold(x[:, i], y)
                 for i in self.get_feature_ids(x.shape[1])])
            best_id = np.argmin(thresholds[:, 0])
            best_thresh = thresholds[best_id, 1]
            self.feature_importances_[best_id] += 1 - thresholds[best_id, 0]
            x_l, x_r, y_l, y_r = self.__div_samples(x, y, best_id, best_thresh)

            if x_l.size == 0 or x_r.size == 0:
                #Set leaf class
                self.tree[node_id] = (self.LEAF_TYPE, np.bincount(y).argmax())
            else:
                self.tree[node_id] = (self.NON_LEAF_TYPE, best_id, best_thresh)
                self.__fit_node(x_l, y_l, 2 * node_id + 1, depth + 1)
                self.__fit_node(x_r, y_r, 2 * node_id + 2, depth + 1)

    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.feature_importances_ = np.zeros(x.shape[1])
        self.__fit_node(x, y, 0, 0)
        self.feature_importances_ /= np.sum(self.feature_importances_)

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
   
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [3]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [4]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1)

## Проверка скорости работы на wine

In [5]:
%time clf.fit(X_train, y_train)

CPU times: user 1.8 ms, sys: 971 µs, total: 2.77 ms
Wall time: 1.82 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [6]:
%time my_clf.fit(X_train, y_train)

CPU times: user 16.6 ms, sys: 1.29 ms, total: 17.9 ms
Wall time: 18.4 ms


## Проверка качества работы на wine

In [7]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.9267399267399267

In [8]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

1.0

In [9]:
my_clf.predict(X_test)

array([1, 0, 2, 1, 0, 2, 0, 2, 2, 0, 2, 2, 2, 0, 0, 0, 0, 1])

## Подготовка данных Speed Dating Data 

In [127]:
#Сделаем преобразования как в примере из лекции

df = pd.read_csv('Speed Dating Data.csv', encoding='latin1')
df = df.iloc[:, :97]
df = df.drop(['id'], axis=1)
df = df.drop(['idg'], axis=1)
df.drop_duplicates(subset=['iid']).gender.value_counts()
df = df.drop(['condtn'], axis=1)
df = df.drop(['round'], axis=1)
df = df.drop(['position', 'positin1'], axis=1)
df = df.drop(['order'], axis=1)
df = df.drop(['partner'], axis=1)

df = df.drop(['age_o', 'race_o', 'pf_o_att',
              'pf_o_sin', 'pf_o_int',
              'pf_o_fun', 'pf_o_amb', 'pf_o_sha',
              'dec_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o',
              'amb_o', 'shar_o', 'like_o', 'prob_o', 'met_o'],
             axis=1)

df = df.dropna(subset=['age'])
df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(0)
df = df.drop(['field'], axis=1)
pd.get_dummies(df, columns=['field_cd'],
               prefix='field_cd', prefix_sep='=')

df = df.drop(['undergra'], axis=1)
df.loc[:, 'mn_sat'] = df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)
df.loc[:, 'mn_sat'] = df.mn_sat.fillna(-999)
df.loc[:, 'tuition'] = df.loc[:, 'tuition'].str.replace(',', 
                                                    '').astype(np.float)
df.loc[:, 'tuition'] = df.tuition.fillna(-999)
df = df.dropna(subset=['imprelig', 'imprace'])
df = df.drop(['from', 'zipcode'], axis=1)
df.loc[:, 'income'] = df.loc[:, 'income'].str.replace(',', '').astype(np.float)
df.loc[:, 'income'] = df.loc[:, 'income'].fillna(-999)
df = df.dropna(subset=['date'])
df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(0)
df = df.drop(['career'], axis=1)

df = df.drop(['sports', 'tvsports', 'exercise',
              'dining', 'museums', 'art', 'hiking',
              'gaming', 'clubbing', 'reading', 'tv',
              'theater', 'movies', 'concerts',
              'music', 'shopping', 'yoga'], axis=1)

df = df.drop(['expnum'], axis=1)
df.loc[:, 'temp_totalsum'] = df.loc[:, [
    'attr1_1', 'sinc1_1', 'intel1_1',
    'fun1_1', 'amb1_1', 'shar1_1'
]].sum(axis=1)

df.loc[:, ['attr1_1', 'sinc1_1',
           'intel1_1', 'fun1_1',
           'amb1_1', 'shar1_1']] = \
    (df.loc[:, ['attr1_1', 'sinc1_1',
                'intel1_1', 'fun1_1',
                'amb1_1', 'shar1_1']].T / df.loc[:, 'temp_totalsum'].T).T * 100

df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr2_1', 'sinc2_1',
                                        'intel2_1', 'fun2_1',
                                        'amb2_1', 'shar2_1']].sum(axis=1)

df.loc[:, ['attr2_1', 'sinc2_1',
           'intel2_1', 'fun2_1',
           'amb2_1', 'shar2_1']] = \
    (df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1',
                'amb2_1', 'shar2_1']].T / df.loc[:, 'temp_totalsum'].T).T * 100

df = df.drop(['temp_totalsum'], axis=1)

for i in [4, 5]:
    feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i),
            'intel{}_1'.format(i), 'fun{}_1'.format(i),
            'amb{}_1'.format(i), 'shar{}_1'.format(i)]

    if i != 4:
        feat.remove('shar{}_1'.format(i))

    df = df.drop(feat, axis=1)

df = df.drop(['wave'], axis=1)
df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid']) \
    .drop(['gender'], axis=1) \
    .dropna()
df_female = df.query('gender == 0').drop_duplicates(subset=['iid']) \
    .drop(['gender', 'match', 'int_corr', 'samerace'], axis=1) \
    .dropna()

df_female.columns = df_female.columns + '_f'
df_female = df_female.drop(['pid_f'], axis=1)
df_pair = df_male.join(df_female.set_index('iid_f'),
                       on='pid',
                       how='inner')
df_pair = df_pair.drop(['iid', 'pid'], axis=1)

X = df_pair.iloc[:, 1:].values
y = df_pair.iloc[:, 0].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [128]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

## Проверка скорости работы на Speed Dating Data 

In [129]:
%time clf.fit(X_train, y_train)

CPU times: user 57.9 ms, sys: 1.18 ms, total: 59.1 ms
Wall time: 58.5 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [130]:
%time my_clf.fit(X_train, y_train)

CPU times: user 585 ms, sys: 3.24 ms, total: 588 ms
Wall time: 588 ms


## Проверка качества работы на Speed Dating Data

In [131]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.51836770436534

In [132]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.5138888888888888

## Задание 3

## Задание 4

In [133]:
df_sk = pd.DataFrame(sorted(list(zip(df_pair.columns[1:], clf.feature_importances_)), key = lambda x: x[-1], reverse=True)[:10])
df_my = pd.DataFrame(sorted(list(zip(df_pair.columns[1:], my_clf.feature_importances_)), key = lambda x: x[-1], reverse=True)[:10])

In [134]:
print("Sklearn clf:")
print(df_sk.to_string(index=False, header=False))
print("\nMy clf:")
print(df_my.to_string(index=False, header=False))

Sklearn clf:
  int_corr  0.083608
 sinc1_1_f  0.036078
  fun1_1_f  0.032701
   shar1_1  0.028222
  mn_sat_f  0.026224
    amb1_1  0.024551
       age  0.023237
  amb1_1_f  0.022374
    race_f  0.021503
  income_f  0.021461

My clf:
   int_corr  0.080229
   samerace  0.080229
   field_cd  0.043081
     goal_f  0.038469
 exphappy_f  0.037368
  sinc1_1_f  0.035546
   fun1_1_f  0.034620
    sinc2_1  0.034544
      age_f  0.034301
    attr2_1  0.031832


## Задание 5

In [135]:
p = {
    'max_depth': range(1, 30),
    'min_samples_leaf': range(1, 20),
    'criterion': ['gini', 'entropy'],
    'n_estimators': range(1, 150)
}
clf = RandomizedSearchCV(RandomForestClassifier(), param_distributions=p, cv=5)
clf.fit(X_train, y_train)
clf.best_params_

{'n_estimators': 76,
 'min_samples_leaf': 2,
 'max_depth': 25,
 'criterion': 'gini'}

In [136]:
best_model = clf.best_estimator_

In [137]:
f1_score(y_pred=best_model.predict(X_test), y_true=y_test, average='macro')

0.5071264367816092